In [8]:
import sys, os
SRC_PATH = os.path.abspath(os.path.join(os.getcwd(), "..",".."))
if SRC_PATH not in sys.path:
    sys.path.insert(0, SRC_PATH)
print(SRC_PATH)

/home/prashant-agrawal/Netflix_Project/src


In [9]:
# ✅ STEP 1: Qdrant Search Path Setup

import os
from utils.path_config import get_base_dir ,get_qdrant_store_path ,get_data_path
from utils.qdrant_client_loader import get_qdrant_client,get_qdrant_collection_name

# Path setup
BASE_DIR = get_base_dir()

qdrant_store_path = get_qdrant_store_path() 

# ✅ Create collection name
COLLECTION_NAME= get_qdrant_collection_name()

print(f"Vector store path: {qdrant_store_path}")
print(f"Base directory: {BASE_DIR}")

Vector store path: /home/prashant-agrawal/Netflix_Project/src/database/qdrant_store_local_db/collection
Base directory: /home/prashant-agrawal/Netflix_Project/src


In [10]:
# qdrant_search_tool.py

import os
from typing import Dict, List, Any
from langchain_core.tools import Tool
from langchain_openai import OpenAIEmbeddings
from langchain_qdrant import QdrantVectorStore
from utils.qdrant_client_loader import get_qdrant_client,get_qdrant_collection_name
from utils.path_config import get_qdrant_store_path

# 🔁 Cache
_qdrant_cache = {}

def qdrant_search_fn(query: str, k: int = 5, filters: Dict[str, Any] = None) -> Dict[str, Any]:
    """Semantic + metadata-filtered search from Qdrant."""
    print("[QDRANT Tool] Query:", query)
    print("[QDRANT Tool] Filters:", filters)

    store_path = get_qdrant_store_path()

    if store_path in _qdrant_cache:
        vectorstore = _qdrant_cache[store_path]
    else:
        if not os.path.exists(store_path):
            print(f"[ERROR] Qdrant store not found at {store_path}")
            return {"results": [], "message": "Qdrant store not found."}

        # ✅ Create embedding + client
        embedding = OpenAIEmbeddings()
        client = get_qdrant_client()  # <-- Singleton loader

        # ✅ Create collection name
        COLLECTION_NAME= get_qdrant_collection_name()

        # ✅ Load Qdrant vector store
        vectorstore = QdrantVectorStore(
            client=client,
            collection_name=COLLECTION_NAME,
            embedding=embedding,
        )
        _qdrant_cache[store_path] = vectorstore

    # ✅ Perform search
    try:
        docs_and_scores = vectorstore.similarity_search_with_score(
            query=query,
            k=k,
            filter=filters
        )
        return {
            "results": [doc.page_content for doc, _ in docs_and_scores],
            "message": f"Found {len(docs_and_scores)} results."
        }
    except Exception as e:
        return {"results": [], "message": str(e)}


In [12]:
# ✅ Test block
if __name__ == "__main__":
    from utils.path_config import get_qdrant_store_path
    from qdrant_client import QdrantClient
    from langchain_qdrant import QdrantVectorStore as Qdrant
    from utils.qdrant_client_loader import get_qdrant_client,get_qdrant_collection_name
    import os

    # Required test input
    query = "Top funded fintech startups in Bengaluru"
    example_filters = {
        "industry_sector": ["Fintech"],
        "headquarters_city": ["bengaluru"],
        "total_funding_raised_inr": {"gte": 50.0}
    }

    # Replace with your real collection name
    COLLECTION_NAME = get_qdrant_collection_name
    
  
    # Run test
    results = qdrant_search_fn(query=query, k=5, filters=example_filters)

    print("\n✅ Qdrant Tool Results:")
    for idx, doc in enumerate(results["results"], 1):
        print(f"{idx}. {doc[:150]}...\n{'-' * 60}")

    print("📦 Message:", results["message"])

[QDRANT Tool] Query: Top funded fintech startups in Bengaluru
[QDRANT Tool] Filters: {'industry_sector': ['Fintech'], 'headquarters_city': ['bengaluru'], 'total_funding_raised_inr': {'gte': 50.0}}
[Qdrant Client] Initializing client with path: /home/prashant-agrawal/Netflix_Project/src/database/qdrant_store_local_db/collection


RuntimeError: Storage folder /home/prashant-agrawal/Netflix_Project/src/database/qdrant_store_local_db/collection is already accessed by another instance of Qdrant client. If you require concurrent access, use Qdrant server instead.